So far we've learnt how to scrape the web, and how to make a request for information from an API. Some websites make APIs even easier. Check out [RapidAPI](https://rapidapi.com/) they take care of writing most of the code for you.

We will use the [AeroDataBox API](https://rapidapi.com/aedbx-aedbx/api/aerodatabox/), which can retrieve all sorts of information about flights and airports. We will show you how to retrieve information about the airports, and then it's up to you to apply this, along with what you've already learnt this week, to **produce a function, which retrieves tomorrows flight information for the major airports in the cities you web scraped**.

In [ ]:
import pandas as pd
import requests

On the left hand side of the AeroDataBox API page, you'll see a list of options for information that you can retrieve:
> - Flights API
- Subsciption / PUSH API
- Airport API
- Aircraft API
- Healthcheck & Status API

1. We want to select `Airport API`

2. Then within Airport API we want to select `Search airports by location`

3. Now in the middle third you'll want to enter the `latitude` and `longitude` of any city to test... we chose Berlin: latitude 52.31 longitude 13.24. Next we changed the `radiusKM` to only 50km. And finally set `withFlightInfoOnly` to true, so it will only return airports which have flight data (scheduled or live) available.

4. On the right hand third of the screen you should see a block of code that looks pretty unfamiliar. This is because by default the code is probably set to *(Node.js) Axios*. However, we have the power to change this to familiar python. Select the dropdown box at the top of the code and select `python > requests`.

Now you can copy the code to your notebook and it should look a little something like the cell below:

In [ ]:
import requests

url = "https://aerodatabox.p.rapidapi.com/airports/search/location/52.31/13.24/km/50/16"

querystring = {"withFlightInfoOnly":"true"}

headers = {
	"X-RapidAPI-Key": "YOUR_API_KEY_HERE",
	"X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

{"items":[{"icao":"EDDB","iata":"BER","name":"Berlin, Berlin Brandenburg","shortName":"Brandenburg","municipalityName":"Berlin","location":{"lat":52.35139,"lon":13.493889},"countryCode":"DE"},{"icao":"EDDT","iata":"TXL","name":"Berlin, Berlin-Tegel","shortName":"-Tegel","municipalityName":"Berlin","location":{"lat":52.5597,"lon":13.287699},"countryCode":"DE"}]}


In [ ]:
import requests

url = "https://aerodatabox.p.rapidapi.com/airports/search/location/52.31/13.24/km/50/16"

querystring = {"withFlightInfoOnly":"true"}

headers = {
	"X-RapidAPI-Key": "YOUR_API_KEY_HERE",
	"X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

{"message":"You are not subscribed to this API."}


Let's view the response as `.json()` instead of `.text` so that it's easier to read

In [ ]:
pd.DataFrame(response.json()['items'])

,icao,iata,name,shortName,municipalityName,location,countryCode
0,EDDB,BER,"Berlin, Berlin Brandenburg",Brandenburg,Berlin,"{'lat': 52.35139, 'lon': 13.493889}",DE
1,EDDT,TXL,"Berlin, Berlin-Tegel",-Tegel,Berlin,"{'lat': 52.5597, 'lon': 13.287699}",DE


We can now turn this into a dataframe using `.json_normalize()`

In [ ]:
pd.json_normalize(response.json()['items'])

,icao,iata,name,shortName,municipalityName,countryCode,location.lat,location.lon
0,EDDB,BER,"Berlin, Berlin Brandenburg",Brandenburg,Berlin,DE,52.35139,13.493889
1,EDDT,TXL,"Berlin, Berlin-Tegel",-Tegel,Berlin,DE,52.55970,13.287699


Let's now use this for the latitude and longitude of multiple cities

In [ ]:
def search_airports(lat, lon):

    url = "https://aerodatabox.p.rapidapi.com/airports/search/location/52.31/13.24/km/50/16"

    querystring = {"withFlightInfoOnly":"true"}

    headers = {
        "X-RapidAPI-Key": "YOUR_API_KEY_HERE",
        "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
    }

    response = requests.request("GET", url, headers=headers, params=querystring)
    
    return pd.json_normalize(response.json()['items'])

In [ ]:
search_airports(52.31, 13.24)

,icao,iata,name,shortName,municipalityName,countryCode,location.lat,location.lon
0,EDDB,BER,"Berlin, Berlin Brandenburg",Brandenburg,Berlin,DE,52.35139,13.493889
1,EDDT,TXL,"Berlin, Berlin-Tegel",-Tegel,Berlin,DE,52.55970,13.287699


In [ ]:
lat = [1, 2, 3 ,4 ,5]
lon = [6, 7, 8, 9, 3]

list_for_df = []

for i in range(len(lat)):
        url = f"https://aerodatabox.p.rapidapi.com/airports/search/location/{lat[i]}/{lon[i]}/km/50/16"

    querystring = {"withFlightInfoOnly":"true"}

    headers = {
        "X-RapidAPI-Key": "YOUR_API_KEY_HERE",
        "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
    }

    response = requests.request("GET", url, headers=headers, params=querystring)
    
    list_for_df.append(pd.json_normalize(response.json()['items']))

  return pd.concat(list_for_df, ignore_index=True)

In [ ]:
def icao_airport_codes(latitudes, longitudes):

  #assert len(latitudes) == len(longitudes)
  
  list_for_df = []

  for i in range(len(latitudes)):

    url = f"https://aerodatabox.p.rapidapi.com/airports/search/location/{latitudes[i]}/{longitudes[i]}/km/100/16"

    querystring = {"withFlightInfoOnly":"true"}

    headers = {
      "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com",
      "X-RapidAPI-Key": "YOUR_API_KEY_HERE"
    }

    response = requests.request("GET", url, headers=headers, params=querystring)

    list_for_df.append(pd.json_normalize(response.json()['items']))

  return pd.concat(list_for_df, ignore_index=True)

###### **Challenge 1:** Icao codes
If you use the above for all of your cities, you can create a DataFrame of all the airports and their associated `icao` codes. Perfect for a relational database.

###### **Challenge 2:** Arrivals information
Using what you have been shown above, plus the skills you've learnt in the last couple of days:
1. In `AeroDataBox API` use the `Flight API` > `FIDS/Schedules: Airport departures and arrivals by airport ICAO code` section
2. Fill out the parameters in the middle third and then copy the `python: requests` code from the right hand third
3. Explore the data you get back. What would be useful in your DataFrame and what can be excluded? Remember Gans wants to know about when people are arriving in the city
4. Make a DataFrame from the information you see as important
5. Condense everything you did above into a function that can take a list of ICAO codes as an input, and as an output gives you a DataFrame with the information for *tomorrows arrivals*

In [ ]:
# your code here